In [1]:

from ultralytics import YOLO
import cv2
import numpy as np
import os
import sys

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd() , '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

video_path = f"{project_root}/data/raw/germany-vs-spain-cutted.mp4"
model_path = f"{project_root}/models/pretrained/yolo11m.pt"
output_path = f"{project_root}/data/object_detected/germany-vs-spain-cutted.mp4"

In [3]:
model = YOLO(model_path)  
    
# Open the video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Could not open video {video_path}")
    

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

In [4]:
# This tells to later save as mp4
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Creates an object that will write video frames into a new file
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height)) 

# A list where we want to save cetner coordinates of detected ball
ball_coordinates = [] 

# Dictionary to store trajectory points for each tracked ball
ball_trajectories = {} 

# Counts the frame we are currently on 
frame_number = 0 

# This checks if the video capture is opened and if its ready
while cap.isOpened(): 

    # Ret is boolean value and true if the frame was read succesfully, frame is an actual image
    ret, frame = cap.read() 

    # This checks if ret is false, and if it is it breaks this 
    if not ret:
        break 

    # Here the results of tracking are being stored
    results = model.track(frame, persist=True) 

    # This writes the current frame into output video file we created (out)
    out.write(frame) 
    
    # Checks if there are any boxes saved in results for this certain frame. results is being overwritten everytime thats why results[0]
    if results[0].boxes is not None: 

        # Here all boxes are being initialized to boxes variable
        boxes = results[0].boxes 

        # Built in python function that checks if an object has a specific attribute, returns True if it has the attr
        # Boxes is an object that contains details. cls is attr we are checking for in boxes
        if hasattr(boxes, 'cls'): 

            # This assigns all detected class names to class_names variable
            class_names = results[0].names 

            # This goes through all classes in boxes 
            for i, cls_id in enumerate(boxes.cls): 

                # This saves boxes id and assigns it to cls_id
                cls_id = int(cls_id) 

                # This gets the name of the class on that specific id
                class_name = class_names[cls_id] 

                # And then this checks if one of those is 'sports ball'
                if class_name == 'sports ball': 

                    # This line gets coordinates of the ball box
                    box = boxes.xyxy[i].cpu().numpy() 

                    # x1, y1, x2, y2 -> is the format in the box and now its getting splitted into variables
                    x1, y1, x2, y2 = box 
                    
                    # Draws a rectangle aroung the detected ball
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2) 

                    # Calculate center point for drawing a point
                    center_x = int((x1 + x2) / 2) 
                    center_y = int((y1 + y2) / 2)
                    
                    # Draws a dot in the middle of the rectangle
                    cv2.circle(frame, (center_x, center_y), 4, (0, 0, 255), -1) 
                    
                    # Store coordinates
                    ball_coordinates.append([frame_number, center_x, center_y])

                    # Get tracking ID (if available)
                    track_id = None
                    if hasattr(boxes, 'id'):
                        track_id = int(boxes.id[i])
                    else:
                        track_id = 0  # Default ID if tracking is not enabled
                    
                    # Add point to trajectory
                    if track_id not in ball_trajectories:
                        ball_trajectories[track_id] = []
                    ball_trajectories[track_id].append((center_x, center_y))

                    # Draw trajectory line
                    if len(ball_trajectories[track_id]) > 1:
                        for i in range(1, len(ball_trajectories[track_id])):
                            # Draw line from previous point to current point
                            cv2.line(frame, 
                                ball_trajectories[track_id][i-1], 
                                ball_trajectories[track_id][i], 
                                (255, 0, 0), 2)

    # This should display a window with the current frame and its named Ball Tracking, this updates on every loop so we can see it in real time happening
    cv2.imshow('Ball Tracking', frame) 

    # This part needs to be added otherwise the window freezes and we get black screen
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

    # Increement frme_number for one to keep track of frame number
    frame_number += 1

# This closes the video capture, if using live webcam than its a must to use realease()
cap.release() 

# This saves the output of the video and finalizes it
out.release() 
cv2.destroyAllWindows()


0: 384x640 9 persons, 1 sports ball, 281.8ms
Speed: 2.3ms preprocess, 281.8ms inference, 5.7ms postprocess per image at shape (1, 3, 384, 640)



qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/denis/Projects/var-ai/venv/lib/python3.11/site-packages/cv2/qt/plugins"


0: 384x640 9 persons, 1 sports ball, 330.0ms
Speed: 12.9ms preprocess, 330.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 1 sports ball, 254.3ms
Speed: 2.3ms preprocess, 254.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 1 sports ball, 1 baseball bat, 287.6ms
Speed: 6.4ms preprocess, 287.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 2 sports balls, 1 baseball bat, 332.4ms
Speed: 2.6ms preprocess, 332.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 2 sports balls, 1 baseball bat, 267.5ms
Speed: 2.1ms preprocess, 267.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 2 sports balls, 1 baseball bat, 256.0ms
Speed: 3.1ms preprocess, 256.0ms inference, 4.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 2 sports balls, 1 baseball bat, 275.3ms
Speed:

In [5]:

# Convert coordinates to numpy array for easier manipulation
ball_coordinates = np.array(ball_coordinates)

# Save coordinates to CSV file
if len(ball_coordinates) > 0:
    coord_path = f"{project_root}/data/coordinates/ger_vs_sp_coordinates.csv"
    np.savetxt(coord_path, ball_coordinates, delimiter=',', 
                header='frame,x,y', comments='', fmt='%d')
    print(f"Ball coordinates saved to {coord_path}")

Ball coordinates saved to /home/denis/Projects/var-ai/data/coordinates/ger_vs_sp_coordinates.csv
